In [97]:
import numpy as np

In [98]:
convert = 1E30*(3.1625E-25)**2
def tdc_coupling(u1,u2,r):
    rMag = np.linalg.norm(r)
    return (np.dot(u1,u2)/(rMag**3) - 3.0*(np.dot(u1,r)*np.dot(u2,r))/(rMag**5))*convert

In [140]:
class freq:
    
    dipoleUnitConvert = 1.0/np.sqrt(42.2561)
    forceConstantUnitConvert = 1E-18
    
    def __init__(self,logFileName):
        log = open(logFileName,"r")
        xyz = []
        self.frequencies = []
        self.ramanIntensities = []
        self.irIntensities = []
        self.dipoleDerivative = []
        self.dipoleMoment = []
        self.atomPositions = []
        self.reducedMasses = []
        self.forceConstant = []
        dipoleDerivativeCount = 0
        normalModeCount = 0
        for line in log:
            if "NAtoms=" in line:
                temp = line.split()
                self.nAtoms = int(temp[1])
            elif "Center     Atomic      Atomic             Coordinates (Angstroms)" in line:
                log.readline()
                log.readline()
                readCoor = "pass"
                atom = 0
                while readCoor == "pass":
                    temp = log.readline()
                    if "-------------------------------------------------------------------" in temp:
                        readCoor = "done"
                    else:
                        temp = temp.split()
                        self.atomPositions.append([])
                        for k in range(3):
                            self.atomPositions[atom].append(float(temp[k+3]))
                        atom += 1
                self.atomPositions = np.array(self.atomPositions)
            elif "Dipole derivatives wrt mode" in line:
                temp = line.split(":")[1].split()
                self.dipoleDerivative.append([])
                for i in range(len(temp)):
                    self.dipoleDerivative[dipoleDerivativeCount].append(float(temp[i].replace('D','E')))
                dipoleDerivativeCount += 1
            elif " Dipole moment (field-independent basis, Debye):" in line:
                temp = log.readline().split()
                for i in range(3):
                    self.dipoleMoment.append(float(temp[i*2+1]))
            elif "Frequencies -- " in line:
                for temp in line.split('--')[1].split():
                    self.frequencies.append(float(temp))
            elif "Red. masses -- " in line:
                for temp in line.split('--')[1].split():
                    self.reducedMasses.append(float(temp))
            elif "Frc consts" in line:
                for temp in line.split('--')[1].split():
                    self.forceConstant.append(float(temp))
            elif "IR Inten    --" in line:
                for temp in line.split('--')[1].split():
                    self.irIntensities.append(float(temp))
            elif "Raman Activ --" in line:
                for temp in line.split('--')[1].split():
                    self.ramanIntensities.append(float(temp))
            elif "Atom  AN      X      Y      Z" in line:
                tempXyz = []
                for atom in range(self.nAtoms):
                    temp = log.readline().split()
                    tempXyz.append([])
                    for i in range(2,len(temp)):
                        tempXyz[atom].append(float(temp[i]))
                tempXyz = np.array(tempXyz)
                if normalModeCount == 0:
                    self.normalModes = np.copy(tempXyz)
                else:
                    self.normalModes = np.column_stack((self.normalModes,tempXyz))
                normalModeCount += 1
        log.close()
        self.nModes = len(self.frequencies)
        self.irIntensities = np.array(self.irIntensities)
        self.frequencies = np.array(self.frequencies)
        self.reducedMasses = np.array(self.reducedMasses)
        self.forceConstant = np.array(self.forceConstant)*self.forceConstantUnitConvert
        vals, self.molecularBasis = np.linalg.eigh(np.dot(self.atomPositions.T,self.atomPositions))
        self.dipoleDerivative = np.array(self.dipoleDerivative)*self.dipoleUnitConvert
        self.dipoleDerivativeMolecularBasis = np.dot(self.dipoleDerivative,self.molecularBasis).T
        self.oscStrength = np.empty(self.nModes,dtype=float)
        for i in range(self.nModes):
            self.oscStrength[i] = np.linalg.norm(self.dipoleDerivative[i,:])

In [141]:
h2oMonomer = freq("h2o_tdc_calc.log")

In [102]:
H0 = np.zeros((6,6),dtype=float)
for i in range(3):
    H0[i,i] = h2oMonomer.forceConstant[i]/h2oMonomer.reducedMasses[i]
    H0[i+3,i+3] = H0[i,i]

In [103]:
H0

array([[1.73152254e-18, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.17103501e-18, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 8.71181423e-18, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.73152254e-18,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.17103501e-18, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 8.71181423e-18]])

In [104]:
h2o1Positions = np.array([[1.499561,    0.481781,   -0.223847],[2.016679,   -0.308853,   -0.005570],[0.941077,    0.617162,    0.560577]])
h2o2Positions = np.array([[-1.190888,   -0.054614,    0.209353],[-0.484323,   -0.083043,   -0.460235],[-1.760870,    0.672061,   -0.083951]])

In [105]:
vals, h2o1MolecularBasis = np.linalg.eigh(np.dot((h2o1Positions-np.mean(h2o1Positions,axis=0)),(h2o1Positions-np.mean(h2o1Positions,axis=0))))
vals, h2o2MolecularBasis = np.linalg.eigh(np.dot((h2o2Positions-np.mean(h2o2Positions,axis=0)),(h2o2Positions-np.mean(h2o2Positions,axis=0))))

In [166]:
V = np.zeros(H0.shape,dtype=float)
for mode1 in range(h2oMonomer.nModes):
    for mode2 in range(h2oMonomer.nModes):
        r = np.mean(h2o2Positions,axis=0) - np.mean(h2o1Positions,axis=0)
        u1 = np.dot(h2o1MolecularBasis.T,h2oMonomer.dipoleDerivativeMolecularBasis[:,mode1]) 
        u2 = np.dot(h2o2MolecularBasis.T,h2oMonomer.dipoleDerivativeMolecularBasis[:,mode2])
        V[mode1 + h2oMonomer.nModes, mode2] = tdc_coupling(u1,u2,r)
        #symmertrize
        V[mode2, mode1 + h2oMonomer.nModes] = V[mode1 + h2oMonomer.nModes, mode2]

In [172]:
V

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.32939815e-21,  1.89561303e-22,  4.00186408e-21],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.10389494e-22,  3.00825630e-23,  5.91024107e-22],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.64801784e-21, -9.90260683e-22,  1.62039274e-21],
       [ 1.32939815e-21,  2.10389494e-22, -6.64801784e-21,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.89561303e-22,  3.00825630e-23, -9.90260683e-22,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.00186408e-21,  5.91024107e-22,  1.62039274e-21,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [177]:
H1 = H0 + V
print(H1)

[[ 1.73152254e-18  0.00000000e+00  0.00000000e+00  1.32939815e-21
   1.89561303e-22  4.00186408e-21]
 [ 0.00000000e+00  8.17103501e-18  0.00000000e+00  2.10389494e-22
   3.00825630e-23  5.91024107e-22]
 [ 0.00000000e+00  0.00000000e+00  8.71181423e-18 -6.64801784e-21
  -9.90260683e-22  1.62039274e-21]
 [ 1.32939815e-21  2.10389494e-22 -6.64801784e-21  1.73152254e-18
   0.00000000e+00  0.00000000e+00]
 [ 1.89561303e-22  3.00825630e-23 -9.90260683e-22  0.00000000e+00
   8.17103501e-18  0.00000000e+00]
 [ 4.00186408e-21  5.91024107e-22  1.62039274e-21  0.00000000e+00
   0.00000000e+00  8.71181423e-18]]


In [174]:
vals, vecs = np.linalg.eigh(H1)

In [175]:
c = 2.99792458e10 # cm-sec-1
energyConvert = 6.02214E46   # convert from J/amu/A^2 to 1/sec^2
tdcFreqs = 1/(2*np.pi*c) * np.sqrt(vals*energyConvert)

In [176]:
tdcFreqs

array([1713.64794255, 1714.96412586, 3724.02472854, 3724.03843981,
       3844.93410066, 3845.64932072])

In [127]:
h2oMonomer.frequencies

array([1714.2948, 3724.0451, 3845.2651])

In [146]:
h2oMonomer.irIntensities[0]

75.5559

In [148]:
u1 = np.matmul(np.concatenate((h2oMonomer.irIntensities,h2oMonomer.irIntensities),axis=0).T,vecs).T

In [155]:
np.outer(u1,u1)

array([[ 1.03149269e+00, -1.08564876e+02, -6.21186805e-01,
         2.46149529e+00, -1.08832066e+00,  2.72363349e+01],
       [-1.08564876e+02,  1.14264818e+04,  6.53800746e+01,
        -2.59073026e+02,  1.14546036e+02, -2.86663140e+03],
       [-6.21186805e-01,  6.53800746e+01,  3.74091889e-01,
        -1.48236475e+00,  6.55409821e-01, -1.64022994e+01],
       [ 2.46149529e+00, -2.59073026e+02, -1.48236475e+00,
         5.87397191e+00, -2.59710633e+00,  6.49952354e+01],
       [-1.08832066e+00,  1.14546036e+02,  6.55409821e-01,
        -2.59710633e+00,  1.14827946e+00, -2.87368650e+01],
       [ 2.72363349e+01, -2.86663140e+03, -1.64022994e+01,
         6.49952354e+01, -2.87368650e+01,  7.19169362e+02]])

In [156]:
oscStrength = np.sqrt(np.diag(np.outer(u1,u1)))

In [157]:
print(oscStrength)

[  1.01562428 106.8947229    0.61163052   2.42362784   1.07157802
  26.81733323]


In [158]:
print(tdcFreqs)

[1707.50013932 1720.66356151 3723.93501714 3724.07296492 3841.83598818
 3848.98630845]


In [159]:
h2oMonomer.irIntensities

array([75.5559,  1.6658, 19.1199])